# Demonstration
End-to-end demonstration.<br>
Ref. https://docs.google.com/document/d/1opi0xGTaMZC0TA2CF2VF1DrWHXeE6M-89ricwlsXtXg/edit?usp=sharing

In [ ]:
import utils.DemoClient as client
import jwt
import json
import boto3
from awscli.customizations.s3.utils import split_s3_bucket_key

## Application Development
Alice uses the Processor Development Environment (PDE) to develop, test and package an application.
Alice's published outputs are:
* Docker image published to DockerHub
* Application Package (CWL) published to Resource Catalogue (TBD) and/or GitHub - accessible by href

## Demo Client Setup
We instantiate a client to interact with the platform.<br>
The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [ ]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "185.52.193.87.nip.io"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

### Authenticated User
We require an authenticated user to interact with some of the platform endpoints - in particular for eoepca v0.3 the ADES.<br>
User authenticates and the client receives an ID Token (JWT) that represents the user.<br>
For convenience within the Jupyter notebook we use a username/password authentication - but the primary mechanism is to rely upon external identity provision.<br>
Currently login with GitHub is supported.

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as user 'demoA' and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="eric"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

## Data Discovery
**TBD - Needs EOX/EOfarm support to elaborate this**<br>
Eric discovers data using platform services (Resource Catalogue / DAS) and QGIS
* OpenSearch endpoint
* Data visualisation

### Resource Catalogue - OpenSearch

In [ ]:
# TBD

### Data Access Services - WMS, WCS (TBD)

In [ ]:
# TBD

## Processing - ADES
The ADES provides WPS 1.0/2.0 and API Processes interfaces - with extensions for process deploy/undeploy.<br>
The ADES provides user-specific endpoints, using a URL path prefix.

In [ ]:
# ADES URLs
ades_base_url = "http://ades." + platform_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

# Init
ades_access_token = None
app_name = "s-expression-0_0_2"

### ADES: List Processes
**GET {service_url}/processes**<br>
Provides a list of all processes 

In [ ]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)
# demo.response_summary(response)

### ADES: Deploy Application
**POST {service_url}/processes**<br>
Deploy application to the ADES.<br>
Request body is json in the same format as defined for an Execute request (ref. API Processes), in which the input is the Application Package describing the application.<br>
The Application Package is a CWL Workflow that is typically provided as an href with content type **_application/atom+xml_** or **_application/cwl_**.

In [ ]:
# API Processes - Deploy Application
response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

**_Dynamic Resource Protection_**<br>
The application deployment creates new endpoints within the ADES that need to be protected - e.g. those for _Details_ **(processes/{application_name})** and _Execute_ **(processes/{application_name}/jobs)**.<br>
During deployment, the ADES interfaces with its PEP (Policy Enforcement Point) to register these endpoints as protected under the ownership of the calling user.

In [ ]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

### ADES: Get Application Details
**GET {service_url}/processes/{application_name}**<br>
Provides details of the deployed application<br>
The response includes the API Processes json application description.

In [ ]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
demo.response_summary(response)

### ADES: Execute Application
**POST {service_url}/processes/{application_name}/jobs**<br>
Request body is json as defined by API Processes to define the inputs and outputs, consistent with the CWL Workflow application package.<br>
The response returns **201 CREATED** to indicate that the job has been successfully initiated.<br>
The response **Location header** provides the path (/processes/{application_name}/jobs/{job_id}) to follow the job status.

In [ ]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/app-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

### ADES: Job Status
**GET {service_url}/watchjob/processes/{application_name}/jobs/{job_id}**<br>
Check the status of a previously submitted job - using the URL returned in the Location header of the execute request.<br>
The response body json provides a status string (success/running/failed) and a % progress.<br>
In the case of a failure then a descriptive message is provided.

In [ ]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
demo.response_summary(response)

**Polling for Job Completion**<br>
The job execution is asynchronous.<br>
The Job Status endpoint is polled until the job completes.

In [ ]:
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Inspect response
demo.response_summary(response)

### ADES Stage-out
At the successful completion of processing, the ADES stages out the results to platform storage (under the direction of the user's Workspace component), and registers the data in the User Workspace for access and further exploitation.<br>
![ADES processing results stage-out](https://raw.githubusercontent.com/EOEPCA/eoepca/develop/technical/workspace/seq-ades-processing-results-stage-out.png)

### ADES: Job Result
**GET {service_url}/watchjob/processes/{application_name}/jobs/{job_id}/result**<br>
Returns details of the outputs for a successful job execution.<br>
The response body provides json data that includes the reference to the STAC file that indexes the processing outputs.

In [ ]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
demo.response_summary(response)
print("stacCatalogUri:", stacCatalogUri)

### ADES: List Jobs
**GET {service_url}/processes/{application_name}/jobs**<br>
Provides a list of all jobs for the named processes 

In [ ]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

### ADES: Undeploy Application
**DELETE {service_url}/processes/{application_name}**<br>
Undeploy application from the ADES

In [ ]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

## Policy Enforcement and Resource Sharing
Owners of resources can choose to share these with other users by updating the access policies

### Deploy Application
The demo user redeploys de application

In [ ]:
# API Processes - Deploy Application
response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check deploy - list processes again
response, process_ids, ades_access_token = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

### UserB Authenticates in the Platform
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as UserB and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="demoB"
USER_PASSWORD="defaultPWD"
userb_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

### UserB Attempts to Execute Demo User Apps
All resources are registered with ownership policies by default

In [ ]:
#-------------------------------------------------------------------------------
# Unauthorized Execution 
# Expected Value: 401
#-------------------------------------------------------------------------------
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/app-execute-body.json", id_token=None, access_token=None)

### Owner Updates Access Policy
Grants access to UserB to execute the deployed app

In [ ]:
#-------------------------------------------------------------------------------
# Get Ownership Id
#-------------------------------------------------------------------------------
owB = demo.get_ownership_id(userb_id_token)
owA = demo.get_ownership_id(user_id_token)

#-------------------------------------------------------------------------------
# Finds the policy by retrieving a resource_id
# Update policy with allowing access to UserB
#-------------------------------------------------------------------------------
ades_res="http://ades.resources.185.52.193.87.nip.io"
res_id = demo.get_resource_by_name(ades_res, "s-expression-0_0_2", user_id_token)
data={'name':'Updated Execution','description':'modified','config':{'resource_id':'res_id','action':'view','rules':[{'OR':[{'EQUAL':{'id':'owA'}},{'EQUAL':{'id':'owB'}}]}]},'scopes':['protected_access']}
pdp_url= "http://test.185.52.193.87.nip.io/pdp"
resp, text= demo.update_policy(pdp_url, data, res_id, user_id_token)

### UserB Executes Application Succesfully
Shared resources can now be executed

In [ ]:
#-------------------------------------------------------------------------------
# User B Execute Application Succesfully
# Expected Value: 201
#-------------------------------------------------------------------------------
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/app-execute-body.json", id_token=userb_id_token, access_token=None)

## Workspace

In [ ]:
# Init
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None
print("workspace_url:", workspace_url)

### Workspace: Get Details

In [ ]:
# Workspace - Get Details
workspace_name = "rm-user-" + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
demo.response_summary(response)

### Inspect S3 Bucket

In [ ]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]
# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)
bucket = s3resource.Bucket(bucket_name)

In [ ]:
stacCatalogUri = 's3://1a85398587cc4b5795dab4ab565dad34:rm-user-eric/wf-d8be29d6-a986-11eb-beca-7258940a7c04/catalog.json'

# Unpick s3 URL
_, stacCatalogKeyName = split_s3_bucket_key(stacCatalogUri)
print("stacCatalogKeyName:", stacCatalogKeyName)

# Read the STAC file
stacFileObject = list(bucket.objects.filter(Prefix=stacCatalogKeyName))[0]
stacFileJson = json.loads(stacFileObject.get()['Body'].read())
print(json.dumps(stacFileJson, indent=2))

## Results Consumption
**TBD - Needs EOX/EOfarm support to elaborate this**<br>
* Results discovery - User's resource catalogue (OpenSearch)
* Results visualisation:
  * Data access services
  * QGIS